# Exercice: Classification d'images de vêtements avec le dataset Fashion-MNIST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thalitadruCoursNNDL/blob/master/ExerciceClassificationFashionMnist.ipynb)

Entraînez des réseaux sur le dataset Fashion-MNIST. Vous devez charger le dataset et l'entraîner avec des modèles différentes que vous allez créer:
1. Un réseau simples avec une couche cachée
1. Un petit réseau CNN


## Imports de modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, losses, optimizers

SEED = 0
np.random.seed(SEED)
rng = np.random.RandomState(SEED)
tf.random.get_global_generator().reset_from_seed(SEED)

plt.style.use("seaborn-talk")

### Fonction pour afficher les courbes d'apprentissage

In [ ]:
def learning_curves(out, **kwargs):
    metrics = out.history.keys()
    metrics = list(filter(lambda m: not m.startswith("val"), metrics))
    num_metrics = len(metrics)
    fig, axs = plt.subplots(
        num_metrics,
        1,
        figsize=(15, 5 * num_metrics),
    )
    for i, metric in enumerate(metrics):
        loss = out.history[metric]
        val_loss = out.history["val_" + metric]
        epochs_range = range(1, len(loss) + 1)
        ax = axs[i]
        ax.plot(
            epochs_range,
            loss,
            marker=".",
            linestyle="dashed",
            label="Train " + metric,
            **kwargs
        )
        ax.plot(
            epochs_range,
            val_loss,
            marker=".",
            linestyle="dashed",
            label="Valid " + metric,
            **kwargs
        )
        ax.legend()
        ax.set_title(metric)
    ax.set_xlabel("epochs")
    fig.suptitle("courbes d'apprentisage x époques", fontsize="x-large")

## Chargement des données
<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

In [ ]:
# ??keras.datasets.fashion_mnist.load_data

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

## Paramètres pour le `compile`

In [ ]:
# compilez le modèle avec les parametres suivants
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = "adam"
metrics = ["accuracy"]

## Partie 1: Réseau simples

- Créez un modèle avec une couche cachée et 50 neurones
    - Vérifiez le `shape` des données. Vous avez peut-être besoin de les aplatir (`layers.Flatten`) avant des les donner à une `layers.Dense`
    - N'oubliez pas d'inclure une étape de rescaling: `tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),`
    - Donnez à chaque couche `Dense` le keyword argument `kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED)`
    - Sauvegardez les poids après création du modèle avec `model.save_weights('init.h5')`
- Compilez le modèle avec un `loss` approprié pour un problème de classification
- Si vous voulez réinitialiser les poids: `model.load_weights('init.h5')`
- Appelez `fit` pour lancer l'entraînement
- Affichez les courbes d'apprentissage pour le loss et l'accuracy

### Ressayez avec 128 neurones
- Refaites le tout avec 128 neurones dans la couche caché.
- Déclarez un nouveau modèle ci-dessous.
- N'oubliez pas de changer le nom de la variable et du fichier `.h5` avec les poids d'initialisation pour ne pas écraser ceux du modèle d'avant.


### Questions
- Pouvez vous dire que ce modèle a plus ou moins de biais que l'autre ?
- Est-ce que ce modèle genéralise mieux que l'autre ?

## Partie 2: Réseau CNN

- Utilisez le modèle ci-dessous
- Compilez le modèle
- Appelez `fit` pour lancer l'entraînement
    - Attention au shape de l'entrée: ici le modèle s'attend a des arrays ndim=4, shape=(28,28,1)
- Affichez les courbes d'apprentissage pour le loss et l'accuracy

In [ ]:
model2 = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
        tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),
        tf.keras.layers.Conv2D(
            filters=16,
            kernel_size=5,
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
        ),
        tf.keras.layers.MaxPool2D(pool_size=2),
        tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=3,
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
        ),
        tf.keras.layers.MaxPool2D(pool_size=2),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(
            units=64,
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
        ),
        tf.keras.layers.Dense(
            units=10,
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
        ),
    ]
)
model2.save_weights("init_cnn.h5")

In [ ]:
model2.summary()

In [ ]:
import visualkeras as vk

vk.layered_view(model2, legend=True, scale_xy=1)

In [ ]:
model2.compile(optimizer, loss, metrics)

In [ ]:
model2.load_weights("init_cnn.h5")
out = out2 = model2.fit(
    tf.expand_dims(train_images, 3),  # rajouter dim de taille 1 à la fin pour avoir (28, 28, 1)
    train_labels,
    epochs=15,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
)

### Questions
- Pouvez vous dire que ce modèle a plus ou moins de biais que les autres ?
- Est-ce que ce modèle généralise mieux que les autres ?
- Combien de fois plus de temps ce modèle CNN à pris pour être entraîné ?

## Comparaison
- Affichez un graphique qui compare leur _accuracy_ en test et en validation.
- En considerant ce critère, quel modèle choisirez vous?

In [ ]:
model = model1 # choisissez model1, model1b ou model2

## Test final
- Évaluez le modèle choisi sur l'ensemble de test

- Cherchez au moins deux images de vêtements sur internet et testez votre modèle dessus: est-ce qu'il sort des prédictions correctes?
Sevez-vous du code ci-dessus:

In [ ]:
# rajoutez d'autres urls d'images et un nom correspondant sur la liste names
urls = [
    "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBQVEhISEhEREhgYERIRERERERERERISGBgZGRkYGRgcIS4lHB4rHxgYJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISGjEhJCQ2NDQ0NDQ0NDE0NDQxNDQ0NDQ0NDQ0NzQxNDE0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDE0P//AABEIAQsAvQMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAACAQMEBQYHAAj/xABJEAACAQICBQcIBgYIBwAAAAABAgADEQQhBRIxUWEGQXGBkaGxBxMiMlJywdEUI2KS0vAWJIKisuEzQkNjZHOTwhU1U1R0g7P/xAAYAQEAAwEAAAAAAAAAAAAAAAAAAQIDBP/EACARAQEAAgEEAwEAAAAAAAAAAAABAhEDEiExURMiQTL/2gAMAwEAAhEDEQA/AOqERDDIgkQGzAYR0wGEBhxGyI+wjLCBm+Xf/LcZ/lW7XUTgjTvPlEbV0XizvFJfvVkHxnBngChyI4zxEdw1K61Gv6qoQN5ZwPC56oJEALRIRiQEiRTEgebmhrBLZBftXv1W+cNYHTfJav6viD/iAOxE+c3yTCeS4fqtb/yj/wDNJu0kJSEjyxpBHlEIGIQiAQgJI9FngIVoE2IYUQiABgGOGNtAbYRsx0xthAyHlPa2i643vh1H+qh+E4XUnafK3iQuBVCGvUxFMC2z0AzG/ZOLPAs9HpfB4xhT1yKmDBqf9JSat26yFX9qVxm88mGFV6WPWogdH+jqysLq2r5xiO9ZfVeSWCdnH0cIFIUajuhuRrEmxz9YDPZaUy5JjdL44WzbkRnp06vyBwp9V8QnAOhHepPfKjSPI3DUQDUxrIGNl16atc8+zmG/YInJjS8eUYYz03LcgRlbFNb/ACQe/XgNyEUbcS/PspqNmftbrx8mPs6MvTItSXzKvf0vPOhH2Aim/aTGll/yi0D9HpUyrFwar67EatiyqFFh7rZ8RKFZaWWbitlnauneS0/q1cf4m/aifKb2mJgfJWfqMSP79TbpRfl3ToNMQH0EfURtBHlElBQIQEUCEBAS0W0W0WBLiGKYMBGjbRxo2YANAMNoBgcg8sOOc4mjh9YaiURWCaufnHZ1uTz5Lluz3znNiSAASSQABmSTkABvmr8o+KL6TxO5DTpL0Ii3/eLSPyDwAq4+nrC60w2IYW51sE/fZT1SLdTaZN3To/JnRC4TDJSv6Z9OsQbg1GAvbgAAB0cZYIltbMG7Fu2PuY0QJx277uqTQG6O6VOmdD08QE84GGodZSjapsdoNxsyEtisYrG3d8Ylsu4WbRyQAAoAAyHVGam/ceJOwj4x8Xt1nmvE9Lm8LQKDlVQLYOrcXsgbo1SGv3Gc1Wdmr4fXVkbYylT0EWnGzTKMyHarMjdKmx8J0cV7aYck77dA8lWI9PFUt606qjoLK3ik6bTnG/J1W1dIIt/XpVU6cg/+ydlpzRmlII8ojSR5ZIMRYgnrwCiXiXiXgTjBMJoBgIY20MwGgCY3aGZF0hiBTo1ah2JSqOf2VLfCB86acr6+KxNS99fE1nB4M7EdxE2vkswoFPE1ucslEcAq6x/iXsnPG2Do2zrHk9oamj6Zt69Sq54+kUHcgmfLfqvxz7NG0GEYE5nSRpFxAz7PjJcYxH4fEwGUWOBYNPYOuPKsBorORcpMP5vGV1tYFtccdYBj3lp2J1nNPKBhtXEU6ntoVO66m/g3dNeK92XJOyFyNfV0hhD/AHhX7yOvxncqU4BoWrqYrDP7OJoE9Gut+6879Tm7BMSOrGEjoMkOXi3giKICz09PQJpgtFiNAAwGhmA0AWma5f4rU0biz7SCkP8A2MqHuYzSGc+8ruM1cNQohrF6xqEb0RT/ALnXsgceqGdr5M09XA4RbW/VqZI4soY95nHMLhWrVqVFc2qVEprbmLEC/Ve/VO3YAAUqYXIBQqjcALCY83iNeLydaNs/COPGma0wbhNTgY24v3bIrVDugo1z1XgOKBDEQEboQtAGpsmL5e4fWwwfnSordTegf4geqbSqJQ8oKGvh6yb6bgdNsu+WwurFMpuOU6xAuNozHSNk+h8LU1lVhsZQw6CLz53XMTuvJSsXwWEYm5+j0wT9pVCnvBnW5mgQx1YzTjywDEKCIYgeAhRBCtAkxDFgwBMAwzAMADMryuwSmpQrHN1V6ag2ICkqxIHMbgTVmZPlJW1qwXmRM/ebM92rMua6xrbgx3nFJoTRiNpHDVnIBQVWQWH1jlCAvUCzfsy7wS2pgbiRKLRuJ+voMP8AuaadrhT3Hvl5Ryaons1ai9QYiYS2469N+SSZ7n6ceMuwG2OvI1QyEGxX4SIcWfpFNbZFGvuudngeyOO3PDoIGWo1s1qUyDznIj/ee2TJtW3SwAhgRFhSFjNQSr0gvoP7reEtnlRpZrU6nunwkxFc4o8msQaS1NVQCgYKzgNbonTuQOsMBSV1KlWqoQdwqOR3GQdJf2dNPYFuIAue4S85OvfD0+Gsp6mPwtNuPkuVu1OXimMli9px9ZHpyQk1c5wQwIIhCSCEWJFgPmJFMSAJgGOGAYAzEadwzpWqXswfXdG389ukZDotNu0gaXwPnqTINUNkyM17Kw6OYi465nyY9WLXhz6ct/jntfS9Ffo70aTg0Wp+eDIEL1EN3ORN77+jdL9MSrvUqJazENYMGsxVSy3GRIJI6pCTkriFZgaaOrteoKdYBX6SQGXql1jsPqMtkWn9VT9BM1Qi4IB5wLCY2Xpu2+WWNs1dmGMiVjJPNIteUEa+0SfoenenUvz1GHUFUSuG2XWih9UDvZz+8Zpx+Wed7Ep7O6FFK2YjjftiylmrpeXcNPKbS63pldlyFv05S5qSo0mgK2ZtUE5sFLau42ERIcIymuqv6y01pqDb0hqMXIG02Krnx4ydyZLatX0SE84ShPOdjW4Cw75Go0WDmsqtUc0vNoEosi3I9fWY88vsJS1ERPZVV6SBmZpxY3e0c+U6dROpySsjJJCzdyHlhiNrHBJCxYkWA+YkKCYCGAYcEwAMSLEMD0qdOrZFfjq/nvltK/TKXpHpHxlM59avj5ihVpGrxzDvcWg1hOV0ILnO8vND/wBCvvP/ABGUbjMy50F/QL71T+IzXj8s8/B2uPT6V+JnjDxS5ofeHaL/AAgmUzmslsL9TNSV2IlhUkDESItVvgh9XT9weEkKIxgvUT3RJKCdU8Oa+T6SQsjrJCyyp1Y4ICwxAWenp6BJniIsQwBtEIhGIYDZEGOMIEBJF0it6bdXjJUZxYvTfolcv5q2PmMZTNnYcY5WjOJyqR0m4nI6UGrt6jL3QS2w9PiXP77fKUNbn6JotEL9RT90ntYma8flnyeD+JS633EEfnrkYyeyXBG8ESBzDojlnfZx3sYqSBiJPqSBW2zOLrjAZ06fuCTUEiaOW1OmPsA9ucnIJ1Tw5r5Ggj6xtBHUEsgYhiIBFEBZ6LPQJM9FnoAkRDCMQiABjZEdgEQBtArr6De6Y7Aq+q3unwkXwmeWH0mlmBgo2Ul6Up3vK1DOT8dRuuNvRNTgEtSpj+7T+ETL4rJSb7eaa6itkT3F8BNeL9Zcg1EgV0sSOJ7DnLFZDxg9I9APd/KTyz6q8d7oNWV1bbLGrslbX2zGN15o1r06fQR2Ej4SeglZohr0xwZh33+Ms6c6sfEcuXlIVY6ojSR1ZZAwIs8IsD09PT0CXPQokATEIh2iGA2REtCIiQAg1fVb3T4RyN1/Uf3G8DIvhMZXHjK/C8pgZeYpbp1ESgO6ckdIMY1x0D5zbkbBwmLRLui7bugPaJtnGc24v1lyfhAJDx65g/Z8CfnJyiRdIDJf2h4S3J/KuH9KypKvEy1eVmLE543qw0FUurruYN2i3wlykz+gzZ2G9fAiaBJ1YX6ufOfZJSOqYyhjiyyp4RRABhAwFnol4t4E60S0KegDEhWiEQAIgMI7aCRAC0CsPQf3W8I7aDVHot7p8JF8EZZxdCJn8QtmM0gG0bxM5jb+cYccpxx1Fo/0lMjmrUz3gzaVBnMXgVvUpjfXpjvUTbOM50cXiseTyQCRtIj0VP2wO0GSlEY0gv1bHcQ3YZbP+arjftFQ0gYpZYNImIXKc0dJnRZtUTpI7QZpFmYwxtUQ/bT+ITTidHFezDk8n0McUxpI4DNGZ0GLeAIt4Bgz14IiwLSeia4ntcQFiRNcT2uN8DxgmKWG+CWG+B60Rth6DPaw3wXYWOfMfCBmm2yl01RswcdBluziM4lAy2O6cLqUWFe1Smd1Sm3Ywm8cZzn9RCji/wDVIYcQDOgOwOYPGdPF+seX8eAnnp6ysu8Edc8CN8cQ8ZrpltmyJHqSdjBao/vGQapnHZq6dc7oFdNs0mEq69NH3qL9Ow995QusstCP6DJ7LXHQ38wZrxXvpTknba3SOCNrHBN2AxFgiLAKLBiwLXKJlOenlHjN1H7j/iifpLjN1H7r/ilPkxX+PJ0PKJlMCnKHFnmpfdf8UL9IcV7NL7r/ADkfLj7T8WTeWESwmCblHi/Ypdj/ADjbcpsX7FLsf5x8uPs+LL035AiWE5+OU2K9il+/84v6S4r2Kf7/AM4+XH2fFl6Wr0gLg3jDUwNjnrBMfwGJapTFRwATra4XZe/N1WMYxLFWDc1rHhxnK3RcbhC63FiRmCNh4S8wFcNSpt9hQekZHvBlStVv6urff/KUON0zWou1OmiOAb3ZiDdszsG+a8eWr3Z549U7N4pEMATnCcqsQP7Gn99vwyVS5W1+ein+o34Zr1xn8daTSrhKh4gEdlvhKp6jk5L2wcHpg1qmrUprTOr6DB9YNbPV2C0stThOfK9+zfGandXFn507DHMBiLVBtAYap6do+I64/igbDVldVGqpYn1buM+cZycLq7RlNzTS03498kL0ntnP6XKtxtpN95ZMTldvo1Oop850dWPtj05em2AO89sMKd57ZjF5ZLz0qo6k/FJCcsU9ip90HwMnqntHTl6a0Kd5i6p3mZZeWVLnFQfsN8If6YUN7/6b/KOqezpvo2UBjT0480BjOWV06NrlHVIP5yjJN8jAGUnRtJKxtk6PnBSqdhjjGVSjolu38iGwBzEW19wlroPRnnHLNmiWuPaJOzotc9m+Wk3dIt1NrLQuiSKV6hKlzrKuVwLZX4nd0RjSujmQHVYPw9VurfsM0LPbM5W2DmEptK4nLLbNrhjpzzPLbLNXfWsqEkbwQAeMpsahLtrG7ax1jxlpiNMU6esCdd/YXb+0di+PCVQcuS+QJJJA2Z80pcZGuNtMhYoAkkJeeNMSu4tqm0a2ee24I2ibHDUnanTe2trorDVBOZFyJj9WafQOnlRFp1MguSVBsA5gw+I/nJkl8oy3J2T/APhtRgbIQN7ej45yq01omoKVQrqsdW1l1iSDt5ts3aG4uM755cZXYyltyNuuaTjkY3krkVNB0SSlOWnKTBrTqB1Hove4HM4+fwMqkfPI9RlMsdNcctw4KfCEKQ6IaOOfKHKrg81wBnvNL0Qrbp7W3+EC+Vp5hzyOla8dDyiQtvgs38oRMbP5EnZogaL5yCeEEnq8JKD+RzE1nJJvq6gO3zncVFvAzFK9obVGuGR3psMtZHZDbjY7Ixy6bsyx6pp0LSFSwmS0xjVRWZjvsOdjzDjKerjsWcji6hG4imx7St5AqprG7lnb2nYs3Vfm4S+XNPxnjw39qrp0Dz7SbniTmZOoAieC/wAjHQJlMm1xSEdWy54rrbiJEcHiDHadc7GgGYKiK9jmICPnnGzTpWg8ctSklj6Sqq1F5wwFr23G2Rk3EvlmBOZI5yKs6kbHRyjr0MM43idIYvZ9LrW6Vv22vNseWa7sMuK77JvLKoLou979gPzmeReueamzMXd3qNs1ncsQNwvs6oocDbcGRct+Fscek4BFFS35yia+/tniZVY6Hia8js9uMcDjojQs9YiOpV2X7fnGW2RaUhKUHniZGXZHuaRUx51jL8Nu7fHAYNTmjZoNjAJIjjRl9hjZp41r5c8bdgYxUig7JSxaUtoaxFipIi1KR+eeNssdfmgPL7UMl7c8JagO3tgPI/OJAsFa081UHbGaRgvJBMg2iNuu+Kpkapt6pMRS2G+OLlx6I0NkQGWVSTY8fGNFbb4kWB//2Q==",
]
names = [
    "pantalon-khaki",
]

In [ ]:
# téléchargement des images choisis
img_paths = []
for url, name in zip(urls, names):
    img_path = tf.keras.utils.get_file(name, origin=url)
    img_paths.append(img_path)

In [ ]:
# evaluation
i = 0
for name, img_path in zip(names, img_paths):
    print("\n{}".format(name))
    img = keras.preprocessing.image.load_img(
        img_path, color_mode = "grayscale", target_size=(28,28)
    )
    plt.style.use("default")
    plt.subplot(3, 3, i + 1)
    plt.imshow(img, cmap='Greys')
    plt.title(name)
    plt.axis("off")
    i += 1

    img_array = keras.preprocessing.image.img_to_array(img)

    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "Image de la classe {} avec  {:.2f}\% de confiance.".format(
            class_names[np.argmax(score)], 100 * np.max(score)
        )
    )